# ***Note:*** cd to your own directory

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Personal projects/NYC Election

Mounted at /content/drive
/content/drive/MyDrive/Personal projects/NYC Election



# **PART 1: NYC Mayoral ENR scrape**
## *— 2025 Mayoral Election Night Results (ENR) by Election District (ED)*

What it does
------------
1) Crawls NYC Board of Elections ENR pages for the given election_id.
2) Extracts ED-level results for ALL Assembly Districts (ADs).
3) Collapses multi-party lines into candidate totals.
4) Adds Total Votes per ED and percentage columns.
5) Writes two CSVs:
   - Raw combined ED results
   - Processed ED results (collapsed + % columns)
6) Downloads NYC Election District (clipped) boundaries for the given release and merges attributes into a GeoDataFrame; writes to GeoPackage

Adjust for future elections
---------------------------
- Change `election_id` (the ENR URL stem) and `nyed_release` (shapefile release quarterly).

Requirements
------------
pip install pandas numpy beautifulsoup4 lxml requests geopandas

Data sources
------------
1) ENR base: https://enr.boenyc.gov/

    Example pages (2025 general; mayor):
    - *County level: https://enr.boenyc.gov/CD27286ADI0.html*
    - *AD level / ED level's link index:     https://enr.boenyc.gov/CD27286AD0.html*
    - *ED level example - AD 23:   https://enr.boenyc.gov/CD27286AD230.html*

2) ED boundaries: NYC DCP (clipped shoreline) release 25c, e.g. nyed_25c.zip
  



## ***Setup***

In [ ]:
# ------------------------------------------
# ------------ Global Variables ------------
# ------------------------------------------
election_id = "CD27286AD"   # ENR URL stem for the race (2025 mayoral)
nyed_release = "25c"        # ED boundary release (e.g., "25c")

In [ ]:
# ------------------------------------------
# ---------------- Imports -----------------
# ------------------------------------------
import os
import re
import zipfile
from io import StringIO
from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup

import shutil
import tempfile


# ------------------------------------------
# --------------- Constants ----------------
# ------------------------------------------
URL_BASE = "https://enr.boenyc.gov/"
URL_SUFFIX = "0.html"

SESSION = requests.Session()
TIMEOUT = 20


# ------------------------------------------
# ----------------- Utils ------------------
# ------------------------------------------
def safe_filename(s: str) -> str:
    """Sanitize string for filenames."""
    return re.sub(r"[^A-Za-z0-9._-]+", "_", s).strip("_")


def fetch(url: str) -> str:
    r = SESSION.get(url, timeout=TIMEOUT)
    r.raise_for_status()
    return r.text

def read_ad_table(ad: str) -> pd.DataFrame:
    """Return raw DataFrame extracted from an AD page table."""
    url = f"{URL_BASE}{election_id}{ad}{URL_SUFFIX}"
    html = fetch(url)
    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", class_="underline")
    if table is None:
        raise RuntimeError(f"Couldn't find ENR results table for AD {ad} ({url})")
    return pd.read_html(StringIO(str(table)))[0]

## ***Get Election Results***

In [ ]:
# ------------------------------------------
# ---------- Get Election Results ----------
# ------------------------------------------

# -------- 1) Get the list of ADs from the AD-level data page --------
ad_url = f"{URL_BASE}{election_id}{URL_SUFFIX}"
html_index = fetch(ad_url)

# Example href pattern: CD27286AD230.html (AD=23)
ad_set = {
    m.group(1)
    for m in re.finditer(rf'href="{re.escape(election_id)}(\d{{2}})0\.html"', html_index)
}
if not ad_set:
    raise RuntimeError("No AD links discovered on the index page. Check election_id.")
ad_list = sorted(ad_set)  # two-digit strings


# ---- 2) Build column headers from any AD page (consistent across pages) ----
# We use the first AD discovered.
df0 = read_ad_table(ad_list[0])

# Keep column 0 (ED), then every other column (skip interstitial NaN columns).
cols_to_keep = [0] + list(range(1, len(df0.columns), 2))
df0 = df0[cols_to_keep]

# Construct headers:
#   - col 0: "ElectDist"
#   - col 1: "Reported"
#   - cols 2..-2: "<Name> <Party>" merged from first two header rows
#   - last col: whatever appears on the page (often "WRITE-IN")
headers = ["ElectDist", "Reported"]
for a, b in zip(df0.iloc[0, 2:-1].astype(str), df0.iloc[1, 2:-1].astype(str)):
    headers.append(f"{a} {b}".strip())
headers.append(str(df0.iloc[0, -1]).strip())


# ---- 3) Extract ED rows for each AD and combine ----
all_ads = []
for ad in ad_list:
    raw = read_ad_table(ad)
    raw = raw[cols_to_keep]
    # drop header rows
    raw = raw.iloc[2:].copy()
    # drop "Total" row (based on first column content)
    raw = raw[raw.iloc[:, 0].astype(str).str.strip().ne("Total")]
    raw.columns = headers

    # Convert ElectDist like "ED 1", "ED 12" -> "AD(2) + ED(3)" as string, e.g., "23001" or "23012"
    raw["ElectDist"] = (
        raw["ElectDist"].astype(str).str.extract(r"(\d+)")[0].astype(int).apply(lambda x: f"{ad}{x:03d}")
    )
    all_ads.append(raw)

combined_df = pd.concat(all_ads, ignore_index=True)

print("Successfully extracted ED-level results into a DataFrame.")


# ---- 4) "Information As Of" timestamp (for filenames) ----
m = re.search(r"Information As Of:\s*([0-9-]{10}\s[0-9:]{8}\s[A-Z]{2,4})", html_index)
info_value = m.group(1) if m else datetime.now().strftime("%Y-%m-%d %H:%M:%S EST")
stamp = safe_filename(info_value)


# ---- 5) Write raw CSV ----
output_folder = 'output_files'
os.makedirs(output_folder, exist_ok=True) # Create the folder if it doesn't exist
raw_csv = f"mayoral_election_results_{stamp}.csv"
combined_df.to_csv(f'{output_folder}/{raw_csv}', index=False)
print(f"Wrote raw CSV: {raw_csv}")

Successfully extracted ED-level results into a DataFrame.
Wrote raw CSV: mayoral_election_results_2025-11-05_00_31_51_EST.csv


## ***Election Results Processing***

In [ ]:
# ------------------------------------------
# ------- Election Results Processing ------
# ------------------------------------------

# ---- 1) Collapse multi-party lines by candidate name ----
# Convert all vote columns to numeric (treat blanks as 0)
non_vote_cols = {"ElectDist", "Reported"}
vote_cols = [c for c in combined_df.columns if c not in non_vote_cols]
numeric_df = combined_df.copy()
numeric_df[vote_cols] = (
    numeric_df[vote_cols]
    .replace({",": ""}, regex=True)
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
    .astype(int)
)

# Collapse multi-party lines by candidate name: strip trailing " (PARTY)"
name_map = {c: re.sub(r"\s*\([^)]*\)\s*$", "", c) for c in numeric_df.columns}
sum_df = numeric_df.rename(columns=name_map)

# Group duplicate candidate columns
# Sum duplicate columns by grouping on axis=1, preserving non-numeric columns.
# Note: This might still trigger a FutureWarning about `groupby(axis=1)` being deprecated,
# but it will correctly sum the columns and populate the DataFrame.
sum_df = sum_df.groupby(sum_df.columns, axis=1).sum()
# ↓ Not Working, returned empty sum_df
# Group duplicate candidate columns (fixes pandas FutureWarning by using T.groupby)
# sum_df = sum_df.T.groupby(level=0, sort=False).sum(numeric_only=True).T


# ---- 2) Compute Total Votes ----
# Compute Total Votes per ED after collapsing
cand_cols = [c for c in sum_df.columns if c not in ("ElectDist", "Reported")]
sum_df["Total Votes"] = sum_df[cand_cols].sum(axis=1)

# Add a grand total row at bottom
total_row = {c: 0 for c in sum_df.columns}
total_row["ElectDist"] = "Total"
total_row["Reported"] = "N/A"
total_row.update(sum_df[cand_cols + ["Total Votes"]].sum(numeric_only=True).to_dict())
sum_df = pd.concat([sum_df, pd.DataFrame([total_row])], ignore_index=True)

# Order candidate columns by their total votes (descending), for readability
totals_series = sum_df.loc[sum_df["ElectDist"].eq("Total"), cand_cols].T.squeeze()
cand_order = totals_series.sort_values(ascending=False).index.tolist()

# Reorder columns
sum_df = sum_df[["ElectDist", "Reported", "Total Votes"] + cand_order]


# ---- 3) Compute Percentage ----
# Percent columns (rounded to 2 decimals, guard against divide-by-zero)
pct_df = sum_df.copy()
for c in cand_order:
    pct_df[f"{c} (%)"] = np.where(
        pct_df["Total Votes"].eq(0), 0.0, (pct_df[c] / pct_df["Total Votes"]) * 100.0
    ).round(2)

# Print final totals row
print("\nGrand totals:")
print(pct_df.tail(1).to_string(index=False))


# ---- 4) Write processed CSV ----
processed_csv = f"processed_mayoral_election_results_{stamp}.csv"
pct_df.to_csv(f'{output_folder}/{processed_csv}', index=False)
print(f"\nWrote processed CSV: {processed_csv}")

/tmp/ipython-input-2748518918.py:26: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  sum_df = sum_df.groupby(sum_df.columns, axis=1).sum()



Grand totals:
ElectDist Reported  Total Votes  Zohran Kwame Mamdani  Andrew M. Cuomo  Curtis A. Sliwa  Eric L. Adams  WRITE-IN  Irene Estrada  Jim Walden  Joseph Hernandez  Zohran Kwame Mamdani (%)  Andrew M. Cuomo (%)  Curtis A. Sliwa (%)  Eric L. Adams (%)  WRITE-IN (%)  Irene Estrada (%)  Jim Walden (%)  Joseph Hernandez (%)
    Total      N/A      2055921               1036051           854995           146137           6382      6290           2643        2170              1253                     50.39                41.59                 7.11               0.31          0.31               0.13            0.11                  0.06

Wrote processed CSV: processed_mayoral_election_results_2025-11-05_00_31_51_EST.csv


## ***Merge with Election Districts Boundaries***

In [ ]:
# ------------------------------------------
# - Merge w/ Election Districts Boundaries -
# ------------------------------------------

# ---- 1) Read ED Boundaries shapefile ----
zip_url = f"https://s-media.nyc.gov/agencies/dcp/assets/files/zip/data-tools/bytes/election-districts/nyed_{nyed_release}.zip"

# Define the subdirectory for shapefiles
shapefile_dir = 'shapefile'
os.makedirs(shapefile_dir, exist_ok=True)

zip_file_path = os.path.join(shapefile_dir, f"nyed_{nyed_release}.zip")
extract_dir_path = os.path.join(shapefile_dir, f"nyed_{nyed_release}")

if not (os.path.exists(zip_file_path) and os.path.exists(extract_dir_path)):
    print("\nDownloading ED boundaries shapefile...")
    with SESSION.get(zip_url, stream=True, timeout=TIMEOUT) as r:
        r.raise_for_status()
        with open(zip_file_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    with zipfile.ZipFile(zip_file_path, "r") as zf:
        zf.extractall(shapefile_dir)

# Find the .shp
shp_path = None
for root, _, files in os.walk(extract_dir_path):
    for fn in files:
        if fn.lower().endswith(".shp"):
            shp_path = os.path.join(root, fn)
            break
    if shp_path:
        break
if not shp_path:
    raise RuntimeError("Shapefile not found in extracted ED boundaries zip.")

ed_gdf = gpd.read_file(shp_path)
if "ElectDist" not in ed_gdf.columns:
    raise RuntimeError("Expected 'ElectDist' column not found in ED shapefile.")
else:
    print("Successfully loaded the NYC Election Districts Boundaries (Clipped to Shoreline) shapefile into a GeoDataFrame.\n")


# ---- 2) Merge with ED results ----
# Merge by ED code (ElectDist is a string like '23001')
ed_gdf["ElectDist"] = ed_gdf["ElectDist"].astype(str)
ed_results_gdf = ed_gdf.merge(pct_df, left_on="ElectDist", right_on="ElectDist", how="left").drop(columns=["Shape_Leng", "Shape_Area"])

# Fill NaNs for numeric outputs
num_cols = ed_results_gdf.select_dtypes(include=[np.number]).columns
if len(num_cols):
    ed_results_gdf[num_cols] = ed_results_gdf[num_cols].fillna(0)
if "Reported" in ed_results_gdf.columns:
    ed_results_gdf["Reported"] = ed_results_gdf["Reported"].fillna("N/A")

print("Successfully merged ED boundaries with ED results.")


# ---- 3) Write GeoPackage ----
# Define paths
gpkg_filename = f"processed_mayoral_election_results_{stamp}.gpkg"
final_gpkg_path = os.path.join(output_folder, gpkg_filename)

# Create a temporary directory and write the GeoPackage there first
with tempfile.TemporaryDirectory() as tmpdir:
    temp_gpkg_path = os.path.join(tmpdir, gpkg_filename)
    ed_results_gdf.to_file(temp_gpkg_path, layer="ed_results", driver="GPKG")

    # Ensure the output directory exists on Google Drive
    os.makedirs(output_folder, exist_ok=True)

    # Move the GeoPackage from the temporary location to the final destination
    shutil.move(temp_gpkg_path, final_gpkg_path)

print(f"\nWrote GeoPackage: {final_gpkg_path}")


Successfully loaded the NYC Election Districts Boundaries (Clipped to Shoreline) shapefile into a GeoDataFrame.

Successfully merged ED boundaries with ED results.

Wrote GeoPackage: output_files/processed_mayoral_election_results_2025-11-05_00_31_51_EST.gpkg


## ***Result***

In [ ]:
ed_results_gdf

,ElectDist,geometry,Reported,Total Votes,Zohran Kwame Mamdani,Andrew M. Cuomo,Curtis A. Sliwa,Eric L. Adams,WRITE-IN,Irene Estrada,Jim Walden,Joseph Hernandez,Zohran Kwame Mamdani (%),Andrew M. Cuomo (%),Curtis A. Sliwa (%),Eric L. Adams (%),WRITE-IN (%),Irene Estrada (%),Jim Walden (%),Joseph Hernandez (%)
0,23001,"POLYGON ((1006386.279 144120.655, 1006261.842 ...",99.00%,1094,78,662,346,1,0,5,2,0,7.13,60.51,31.63,0.09,0.00,0.46,0.18,0.00
1,23002,"POLYGON ((1009207.47 145106.094, 1009147.042 1...",99.00%,1078,75,627,369,1,0,4,1,1,6.96,58.16,34.23,0.09,0.00,0.37,0.09,0.09
2,23003,"POLYGON ((1022349.755 145817.949, 1022404.165 ...",99.00%,376,33,184,157,0,1,0,0,1,8.78,48.94,41.76,0.00,0.27,0.00,0.00,0.27
3,23004,"POLYGON ((1025161.154 147197.743, 1025055.81 1...",99.00%,1092,102,757,226,3,2,2,0,0,9.34,69.32,20.70,0.27,0.18,0.18,0.00,0.00
4,23005,"POLYGON ((1026261.273 147727.118, 1026236.365 ...",99.00%,1091,82,731,274,1,3,0,0,0,7.52,67.00,25.11,0.09,0.27,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4259,53009,"POLYGON ((1009016.656 194699.044, 1008704.293 ...",99.00%,581,486,65,28,1,0,0,1,0,83.65,11.19,4.82,0.17,0.00,0.00,0.17,0.00
4260,74036,"POLYGON ((989457.788 203440.679, 989341.403 20...",99.00%,761,585,148,22,0,3,2,1,0,76.87,19.45,2.89,0.00,0.39,0.26,0.13,0.00
4261,65057,"POLYGON ((989093.93 202786.694, 988972.11 2025...",99.00%,955,743,178,27,1,4,0,1,1,77.80,18.64,2.83,0.10,0.42,0.00,0.10,0.10
4262,53012,"POLYGON ((1008085.86 196047.279, 1008291.011 1...",85.71%,375,299,63,11,0,0,1,1,0,79.73,16.80,2.93,0.00,0.00,0.27,0.27,0.00


## **(TEST) Check the ElectDist difference**
### ***Conclusion:*** The election results data for Elect Dists appeared in the election night results but not in the Elect Dists boundary shapefile are all values 0

In [ ]:
# Get the unique values from 'ElectDist' in pct_df
unique_pct_df = set(pct_df['ElectDist'].unique())

# Get the unique values from 'ElectDist' in ed_gdf and convert to strings
unique_ed_gdf = set(ed_gdf['ElectDist'].astype(str).unique())

# Find the difference: elements in pct_df['ElectDist'] but not in ed_gdf['ElectDist']
diff_pct_df_not_ed_gdf = unique_pct_df - unique_ed_gdf

# Find the difference: elements in ged_gdf['ElectDist'] but not in pct_df['ElectDist']
diff_ed_gdf_not__pct_df = unique_ed_gdf - unique_pct_df

print("Election Districts in pct_df but not in ed_gdf:")
print(diff_pct_df_not_ed_gdf)

print("\nElection Districts in ed_gdf but not in pct_df:")
print(diff_ed_gdf_not__pct_df)

print(f"\nNumber of unique ElectDist in pct_df: {len(unique_pct_df)}")
print(f"Number of unique ElectDist in ed_gdf: {len(unique_ed_gdf)}")

Election Districts in pct_df but not in ed_gdf:
{'76063', '45056', '67070', '67071', '52091', '26062', '64058', '74067', '64057', '72064', '72065', '66077', '46075', '64054', '35055', '63071', '76066', '85058', '85063', '68078', '61070', '71066', '23073', '69081', '72066', '64055', '85055', '85062', '64088', '63076', '81060', '61112', '63070', '52094', '23067', '70072', '68080', '76064', '62063', '71068', '35059', '36068', '37070', '31058', '85064', '61072', '36069', '46074', '66076', '64090', '65068', '46078', '74069', '69084', '61110', '75069', '46076', '66075', '71070', 'Total', '71069', '61102', '27064', '69078', '61101', '59068', '69082', '69080', '61114', '27065', '65071', '70073', '61111', '61109', '59070'}

Election Districts in ed_gdf but not in pct_df:
set()

Number of unique ElectDist in pct_df: 4339
Number of unique ElectDist in ed_gdf: 4264


In [ ]:
# Filter pct_df to include only rows where 'ElectDist' is in diff_pct_df_not_ed_gdf
subset_diff_pct_df_not_ed_gdf = pct_df[pct_df['ElectDist'].isin(diff_pct_df_not_ed_gdf)]

display(subset_diff_pct_df_not_ed_gdf)

,ElectDist,Reported,Total Votes,Zohran Kwame Mamdani,Andrew M. Cuomo,Curtis A. Sliwa,Eric L. Adams,WRITE-IN,Irene Estrada,Jim Walden,Joseph Hernandez,Zohran Kwame Mamdani (%),Andrew M. Cuomo (%),Curtis A. Sliwa (%),Eric L. Adams (%),WRITE-IN (%),Irene Estrada (%),Jim Walden (%),Joseph Hernandez (%)
66,23067,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
72,23073,66.67%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
250,26062,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
317,27064,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
318,27065,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4207,85058,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4211,85062,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4212,85063,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4213,85064,99.00%,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
# Select columns that represent vote counts (excluding 'ElectDist' and 'Reported')
vote_columns = subset_diff_pct_df_not_ed_gdf.columns.difference(['ElectDist', 'Reported'])

# Convert vote columns to numeric, coercing errors and filling NaN with 999
for col in vote_columns:
    subset_diff_pct_df_not_ed_gdf.loc[:, col] = pd.to_numeric(subset_diff_pct_df_not_ed_gdf[col], errors='coerce').fillna(999)

# Check if all values in vote columns are zero for each row
all_zeros = (subset_diff_pct_df_not_ed_gdf[vote_columns] == 0).all(axis=1)

# Count how many rows have all zeros in vote columns
num_all_zeros = all_zeros.sum()

print(f"Number of EDs in subset_diff_pct_df_not_ed_gdf with all vote counts as zero: {num_all_zeros}")

# Display rows that do NOT have all zeros (if any)
if num_all_zeros < len(subset_diff_pct_df_not_ed_gdf):
    print("\nEDs in subset_diff_pct_df_not_ed_gdf with non-zero vote counts:")
    display(subset_diff_pct_df_not_ed_gdf[~all_zeros])
else:
    print("\nAll EDs in subset_diff_pct_df_not_ed_gdf have zero vote counts.")

Number of EDs in subset_diff_pct_df_not_ed_gdf with all vote counts as zero: 74

EDs in subset_diff_pct_df_not_ed_gdf with non-zero vote counts:


,ElectDist,Reported,Total Votes,Zohran Kwame Mamdani,Andrew M. Cuomo,Curtis A. Sliwa,Eric L. Adams,WRITE-IN,Irene Estrada,Jim Walden,Joseph Hernandez,Zohran Kwame Mamdani (%),Andrew M. Cuomo (%),Curtis A. Sliwa (%),Eric L. Adams (%),WRITE-IN (%),Irene Estrada (%),Jim Walden (%),Joseph Hernandez (%)
4338,Total,N/A,2055921,1036051,854995,146137,6382,6290,2643,2170,1253,50.39,41.59,7.11,0.31,0.31,0.13,0.11,0.06


# **PART 2: Map the Result**

## Map 1: Mamdani Only

In [ ]:
import folium

# Ensure the GeoDataFrame is in a geographic coordinate system (e.g., WGS84 for Folium)
# If it's not, reproject it:
if ed_results_gdf.crs and ed_results_gdf.crs.to_epsg() != 4326:
    ed_results_gdf = ed_results_gdf.to_crs(epsg=4326)

# Calculate the centroid to center the map using union_all() as recommended
center_point = ed_results_gdf.geometry.union_all().centroid

# Create a Folium map centered on the area
m = folium.Map(location=[center_point.y, center_point.x], tiles="Cartodb Positron", zoom_start=11)

# Get candidate percentage columns for tooltips
candidate_pct_cols = [col for col in ed_results_gdf.columns if col.endswith(' (%)')]

# Define columns for the tooltip
tooltip_cols = ['ElectDist', 'Total Votes'] + candidate_pct_cols

# Add choropleth layer to the map
folium.Choropleth(
    geo_data=ed_results_gdf,
    name='Choropleth',
    data=ed_results_gdf,
    columns=['ElectDist', 'Zohran Kwame Mamdani (%)'], # Column to color the map by
    key_on='feature.properties.ElectDist',
    fill_color='YlGnBu', # Color scheme
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color="red",
    nan_fill_opacity=0.4,
    legend_name='Zohran Kwame Mamdani (%)',
    highlight=True # Highlight on hover
).add_to(m)

# Add tooltips for each feature
folium.features.GeoJson(
    ed_results_gdf.to_json(),
    name='ED Results Tooltips',
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols,
        aliases=[col.replace(' (%)', '') for col in tooltip_cols], # Clean up aliases
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    ),
    style_function=lambda x: {'color': 'grey', 'weight': 0.5}, # Add style function for borders
    highlight_function=lambda x: {'fillColor': '#d35400', 'color': '#d35400', 'weight': 1, 'dashArray': ''}
).add_to(m)

# Display the map
m

Output hidden; open in https://colab.research.google.com to view.

## Map 2: Mamdani vs. Cuomo

### Prepare Data for Mapping

Calculate the winning candidate and their percentage for each Election District and add these as new columns to a copy of the 'ed_results_gdf' GeoDataFrame. This involves identifying the candidate with the highest percentage of votes for each ED.


In [ ]:
# Copy ed_results_gdf
ed_map_gdf = ed_results_gdf.copy()

# Find candidate pct columns
candidate_pct_columns = [col for col in ed_map_gdf.columns if col.endswith(' (%)')]
print(f"Identified {len(candidate_pct_columns)} candidate percentage columns.")

# Determine the winning candidate and their percentage
# Find the column with the maximum percentage value for each row within the `candidate_pct_columns` and store both the column name and the maximum value in new temporary columns
ed_map_gdf['temp_winning_candidate_col'] = ed_map_gdf[candidate_pct_columns].idxmax(axis=1)
ed_map_gdf['winning_percentage'] = ed_map_gdf[candidate_pct_columns].max(axis=1)
ed_map_gdf['winning_candidate'] = ed_map_gdf['temp_winning_candidate_col'].str.replace(' (%)', '', regex=False)
print("Calculated winning candidate and percentage for each ED.")

# For districts with no reported votes
# Identify rows where 'Total Votes' is 0 and set the 'winning_candidate' to 'No Votes' and 'winning_percentage' to 0.0 in those rows
no_votes_mask = ed_map_gdf['Total Votes'] == 0
ed_map_gdf.loc[no_votes_mask, 'winning_candidate'] = 'No Votes'
ed_map_gdf.loc[no_votes_mask, 'winning_percentage'] = 0.0
print("Updated winning candidate and percentage for districts with no votes.")

Identified 8 candidate percentage columns.
Calculated winning candidate and percentage for each ED.
Updated winning candidate and percentage for districts with no votes.


### Define Colormaps

Import the 'branca.colormap' module and define two colormaps: 'OrRd' for Zohran Kwame Mamdani and 'GnBu' for Andrew M. Cuomo, scaled from 0 to 100 for percentage representation, and add captions for the legends.


In [ ]:
import branca.colormap as cm

# Define colormap for Zohran Kwame Mamdani
mamdani_colormap = cm.linear.OrRd_04.scale(vmin=0, vmax=100)
mamdani_colormap.caption = 'Zohran Kwame Mamdani (%)'
print(f"Defined mamdani_colormap with caption: {mamdani_colormap.caption}")

# Define colormap for Andrew M. Cuomo
cuomo_colormap = cm.linear.GnBu_04.scale(vmin=0, vmax=100)
cuomo_colormap.caption = 'Andrew M. Cuomo (%)'
print(f"Defined cuomo_colormap with caption: {cuomo_colormap.caption}")

Defined mamdani_colormap with caption: Zohran Kwame Mamdani (%)
Defined cuomo_colormap with caption: Andrew M. Cuomo (%)


### Create Custom Style Function

A Python function that determines the fill color, fill opacity, line color, line weight, and dash array for each GeoJSON feature based on the winning candidate. It uses the defined colormaps for Mamdani and Cuomo, and a red dashed line for all other winners or EDs with zero total votes.


This function takes a GeoJSON feature as input and, based on the `winning_candidate` and `winning_percentage` in its properties, it returns a dictionary of styling attributes (fill color, opacity, line color, weight, and dash array).


In [ ]:
def style_function(feature):
    winning_candidate = feature['properties']['winning_candidate']
    winning_percentage = feature['properties']['winning_percentage']

    if winning_candidate == 'Zohran Kwame Mamdani':
        fill_color = mamdani_colormap(winning_percentage)
        fill_opacity = 0.7
        line_color = 'black'
        line_weight = 0.5
        dash_array = ''
    elif winning_candidate == 'Andrew M. Cuomo':
        fill_color = cuomo_colormap(winning_percentage)
        fill_opacity = 0.7
        line_color = 'black'
        line_weight = 0.5
        dash_array = ''
    else:
        fill_color = 'transparent'
        fill_opacity = 0.4
        line_color = 'red'
        line_weight = 0.5
        dash_array = '5, 5'

    return {
        'fillColor': fill_color,
        'fillOpacity': fill_opacity,
        'color': line_color,
        'weight': line_weight,
        'dashArray': dash_array
    }
print("Defined custom style_function for mapping.")

Defined custom style_function for mapping.


### Generate Folium Map

Create a Folium map centered on the NYC area. Use 'folium.features.GeoJson' with the custom style function to render the Election Districts with the conditional coloring. Configure the tooltips to display relevant election results, including the winning candidate and percentage. Add the colormap legends to the map.


In [ ]:
import folium

# Ensure the GeoDataFrame is in a geographic coordinate system (e.g., WGS84 for Folium)
# If it's not, reproject it:
if ed_map_gdf.crs and ed_map_gdf.crs.to_epsg() != 4326:
    ed_map_gdf = ed_map_gdf.to_crs(epsg=4326)

# Calculate the centroid to center the map using union_all() as recommended
center_point = ed_map_gdf.geometry.union_all().centroid

# Create a Folium map centered on the area
m = folium.Map(location=[center_point.y, center_point.x], tiles="Cartodb Positron", zoom_start=11)

# Define columns for the tooltip
tooltip_cols = [
    'ElectDist', 'Total Votes', 'winning_candidate', 'winning_percentage',
    'Zohran Kwame Mamdani (%)', 'Andrew M. Cuomo (%)', 'Curtis A. Sliwa (%)',
    'Eric L. Adams (%)', 'WRITE-IN (%)', 'Irene Estrada (%)', 'Jim Walden (%)', 'Joseph Hernandez (%)'
]

# Add GeoJson layer with custom styling and tooltips
folium.GeoJson(
    ed_map_gdf.__geo_interface__,
    name='Election Districts',
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=tooltip_cols,
        aliases=[col.replace(' (%)', '') for col in tooltip_cols], # Clean up aliases
        localize=True,
        sticky=False,
        labels=True,
        max_width=800,
    )
).add_to(m)

# Add colormap legends to the map
mamdani_colormap.add_to(m)
cuomo_colormap.add_to(m)

# Add a layer control to toggle layers if desired
folium.LayerControl().add_to(m)

# Display the map
m

Output hidden; open in https://colab.research.google.com to view.

## Export

Export ed_map_gdf as CSV and GeoPkg, export m as HTML

In [ ]:
csv_output_path = os.path.join(output_folder, f"ed_map_gdf_results_{stamp}.csv")
ed_map_gdf.to_csv(csv_output_path, index=False)
print(f"Exported ed_map_gdf to CSV: {csv_output_path}")

gpkg_output_path = os.path.join(output_folder, f"ed_map_gdf_results_{stamp}.gpkg")
ed_map_gdf.to_file(gpkg_output_path, driver="GPKG")
print(f"Exported ed_map_gdf to GeoPackage: {gpkg_output_path}")

html_output_path = os.path.join(output_folder, f"mayoral_election_map_{stamp}.html")
m.save(html_output_path)
print(f"Exported Folium map to HTML: {html_output_path}")

Exported ed_map_gdf to CSV: output_files/ed_map_gdf_results_2025-11-05_00_31_51_EST.csv
Exported ed_map_gdf to GeoPackage: output_files/ed_map_gdf_results_2025-11-05_00_31_51_EST.gpkg
Exported Folium map to HTML: output_files/mayoral_election_map_2025-11-05_00_31_51_EST.html
